In [ ]:
import torch
import numpy as np
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
model = torch.load("cswin_small_224.pth", map_location='cpu')['state_dict_ema']

In [ ]:
model_keys = []
nums = 0.
for key in model.keys():
    if "num_batches_tracked" not in key:
        model_keys.append(key)
        nums += model[key].numel()
        print(key, model[key].shape)
print(nums)

In [ ]:
weights = []
prefix = "model."
k = 0
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    else:
        data_type = dtype.float32
        weight = Parameter(Tensor(model[key].numpy(), data_type),requires_grad=True)
        temp = prefix + key
        if "stage1_conv_embed.0" in temp:
            temp = temp.replace("0.", "")
        if "stage1_conv_embed.2" in temp:
            temp = temp.replace("stage1_conv_embed.2", "stage1_conv_norm")
            temp = temp.replace(".weight", ".gamma")
            temp = temp.replace(".bias", ".beta")
        if "attns" in temp:
            temp = temp.replace("attns", "attns.")
        print(temp)
        k+=1
        weights.append({"name": temp, "data": weight})
#     print(weights[-1]['name'])
save_checkpoint(weights, "cswin_small_224.ckpt")

In [ ]:
len(weights)